# GFK

In [1]:
#adaptacion
from utils.adaptacion import adapt_gfk, transform_gfk

#clasificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from utils.clasificacion import *

#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#otros
import os
import numpy as np
import pandas as pd

#variables para guardar los resultados
tipo = pruebas[2]
dataset_name = datasets[1]
dims = dimensions[dataset_name]

Using Theano backend.
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes an

In [2]:
def compute_accuracy(K, Xs, Ys, Xt, Yt):
    np.random.seed(10)

    source = np.diag(np.dot(np.dot(Xs, K), Xs.T))
    source = np.reshape(source, (Xs.shape[0], 1))
    source = np.matlib.repmat(source, 1, Yt.shape[0])
    
    target = np.diag(np.dot(np.dot(Xt, K), Xt.T))
    target = np.reshape(target, (Xt.shape[0], 1))
    target = np.matlib.repmat(target, 1, Ys.shape[0]).T
    
    dist = source + target - 2 * np.dot(np.dot(Xs, K), Xt.T)
    
    indices = np.argmin(dist, axis=0)
    prediction = Ys[indices]
    
    accuracy = sum(prediction == Yt) / float(Yt.shape[0])
    
    return accuracy

def compute_knn(K, Xs, Ys, Xt, Yt):
    Xs = np.dot(Xs, K)
    Xt = np.dot(Xt, K)
    
    neigh = KNeighborsClassifier().fit(Xs, Ys)
    
    return neigh.score(Xt, Yt)

## Pruebas con el dataset Twitter

In [3]:
print tipo
print dataset_name
print dims
print data_path

gfk
twitter
2000
data


In [4]:
# cargando dataset Twitter
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

In [5]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 2" % (i+1)
            
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #############
            #### GFK ####
            #############
            
            
            X_tr = np.asarray(labeled[src]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se adaptan los dominios usando GFK
            print "Adaptando dominios..."
            gfk = adapt_gfk(X_tr, X_ts, 10, dims/2)
            
            t_error = compute_accuracy(gfk.G, X_tr, y_tr, X_ts, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['GFK',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i += 1
            
print "Pruebas completadas."

Tarea 1 de 2
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 2
Cargando modelo existente.
Adaptando dominios...
Pruebas completadas.


In [6]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,GFK,r->t,rio2016,thevoice,16.081594,58.461538,42.379945
1,GFK,t->r,thevoice,rio2016,20.687361,50.000000,29.312639


In [7]:
new_scores_path = os.path.join(scores_path, tipo, dataset_name+".csv")

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/gfk/twitter.csv
Resultados guardados.


# Arreglar esto

In [ ]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 2" % (i+1)
            
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #############
            #### GFK ####
            #############
            
            
            X_tr = np.asarray(labeled[src]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se adaptan los dominios usando GFK
            print "Adaptando dominios..."
            gfk = adapt_gfk(X_tr, X_ts, 10, dims/2)
            
            print "Transformando dominios..."
            X_tr_a = transform_gfk(X_tr, gfk)
            X_ts_a = transform_gfk(X_ts, gfk)
            
            print "Entrenando clasificador..."
            svc_a = SVC(kernel='linear')
            svc_a = svc_a.fit(X_tr_a, y_tr)
            
            print "Probando el clasificador"
            t_error = 1-svc_a.score(X_ts_a, y_ts)
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['GFK',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i += 1
            
print "Pruebas completadas."